In [1]:
import numpy as np

from tvm import relay
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_executor
import tvm.testing

In [2]:
bs = 1
num_class = 1000

img_shape = (3, 224, 224)

data_shape = (bs, ) + img_shape
output_shape = (bs, num_class)

In [3]:
mod, params = relay.testing.resnet.get_workload(
    num_layers=18, batch_size=bs, image_shape=img_shape
)

In [7]:
# set show_meta_data=True if you want to show meta data
print(mod.astext(show_meta_data=False))

#[version = "0.0.5"]
def @main(%data: Tensor[(1, 3, 224, 224), float32], %bn_data_gamma: Tensor[(3), float32], %bn_data_beta: Tensor[(3), float32], %bn_data_moving_mean: Tensor[(3), float32], %bn_data_moving_var: Tensor[(3), float32], %conv0_weight: Tensor[(64, 3, 7, 7), float32], %bn0_gamma: Tensor[(64), float32], %bn0_beta: Tensor[(64), float32], %bn0_moving_mean: Tensor[(64), float32], %bn0_moving_var: Tensor[(64), float32], %stage1_unit1_bn1_gamma: Tensor[(64), float32], %stage1_unit1_bn1_beta: Tensor[(64), float32], %stage1_unit1_bn1_moving_mean: Tensor[(64), float32], %stage1_unit1_bn1_moving_var: Tensor[(64), float32], %stage1_unit1_conv1_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_bn2_gamma: Tensor[(64), float32], %stage1_unit1_bn2_beta: Tensor[(64), float32], %stage1_unit1_bn2_moving_mean: Tensor[(64), float32], %stage1_unit1_bn2_moving_var: Tensor[(64), float32], %stage1_unit1_conv2_weight: Tensor[(64, 64, 3, 3), float32], %stage1_unit1_sc_weight: Tensor[(64, 64, 1

In [19]:
opt_level = 3
target = tvm.target.cuda()
with tvm.transform.PassContext(opt_level=opt_level):
    lib = relay.build(mod, target, params=params)
    # _, mylib, _ = relay.build(mod, target, params=params)


/tmp/ipykernel_76629/2282994270.py:5: DeprecationWarning: legacy graph executor behavior of producing json / lib / params will be removed in the next release. Please see documents of tvm.contrib.graph_executor.GraphModule for the  new recommended usage.
  _, mylib, _ = relay.build(mod, target, params=params)


In [10]:
dev = tvm.cuda()
data = np.random.uniform(-1, 1, size=data_shape).astype("float32")
module = graph_executor.GraphModule(lib["default"](dev))

In [11]:
module.set_input("data", data)

In [12]:
module.run()

In [16]:
out = module.get_output(0, tvm.nd.empty(output_shape)).numpy()


In [17]:
print(out.flatten()[0:10])


[0.00089283 0.00103331 0.0009094  0.00102275 0.00108751 0.00106737
 0.00106262 0.00095838 0.00110792 0.00113151]


In [20]:
import tvm
import tensorflow as tf
import numpy as np
import tvm.relay as relay

2022-04-05 15:09:58.999490: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [21]:
model_keras = tf.keras.models.load_model('./very_simple_model.h5')

input_data = np.random.normal(0,1,(1,256,256,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


2022-04-05 15:10:09.481709: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-05 15:10:09.481773: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-04-05 15:10:09.481863: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 15:10:09.482254: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-04-05 15:10:09.482270: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-05 15:10:09.483948: I tensorflow/stream_executor/platfor

In [1]:
import numpy as np
import tvm
from tvm import te
import tvm.relay as relay

In [2]:
def example():
    shape = (1, 64, 54, 54)
    c_data = np.empty(shape).astype("float32")
    c = relay.const(c_data)
    weight = relay.var("weight", shape=(64, 64, 3, 3))
    x = relay.var("x", relay.TensorType((1, 64, 56, 56), "float32"))
    conv = relay.nn.conv2d(x, weight)
    y = relay.add(c, c)
    y = relay.multiply(y, relay.const(2, "float32"))
    y = relay.add(conv, y)
    z = relay.add(y, c)
    z1 = relay.add(y, c)
    z2 = relay.add(z, z1)
    return relay.Function([x, weight], z2)

In [12]:
# Let's first create a relay Module which contains one or multiple Relay
# functions for optimization.
f = example()
mod = tvm.IRModule.from_expr(f)

In [13]:
print(mod.astext(show_meta_data=False))

#[version = "0.0.5"]
def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) {
  %0 = add(meta[relay.Constant][0], meta[relay.Constant][0]);
  %1 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]);
  %2 = multiply(%0, 2f);
  %3 = add(%1, %2);
  %4 = add(%3, meta[relay.Constant][0]);
  %5 = add(%3, meta[relay.Constant][0]);
  add(%4, %5)
}

/* For debugging purposes the metadata section has been omitted.
 * If you would like to see the full metadata section you can set the 
 * option to `True` when invoking `astext`. 
 */


In [14]:
fold_const = relay.transform.FoldConstant()
# Then, we can invoke the pass on the given module. Note that the constant
# folding pass works at the function-level. That being said, each function in
# the module will be applied with the optimization. Users don't need to iterate
# through individual functions manually to apply this pass.
mod1 = fold_const(mod)
# We can see from the updated program that the constants are folded.
print(mod1)


def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) -> Tensor[(1, 64, 54, 54), float32] {
  %0 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %1 = add(%0, meta[relay.Constant][0] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %2 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %3 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  add(%2, %3) /* ty=Tensor[(1, 64, 54, 54), float32] */
}




In [15]:
mod2 = relay.transform.EliminateCommonSubexpr()(mod1)
print(mod2)


def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) -> Tensor[(1, 64, 54, 54), float32] {
  %0 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %1 = add(%0, meta[relay.Constant][0] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %2 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  add(%2, %2) /* ty=Tensor[(1, 64, 54, 54), float32] */
}




In [10]:
mod = relay.transform.FuseOps(fuse_opt_level=0)(mod)

# We can observe that the optimized module contains functions that only have
# a signle primitive op.
print(mod)

def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) -> Tensor[(1, 64, 54, 54), float32] {
  %0 = fn (%p03: Tensor[(1, 64, 56, 56), float32], %p12: Tensor[(64, 64, 3, 3), float32], Primitive=1) -> Tensor[(1, 64, 54, 54), float32] {
    nn.conv2d(%p03, %p12, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 64, 54, 54), float32] */
  };
  %1 = %0(%x, %weight) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %2 = fn (%p02: Tensor[(1, 64, 54, 54), float32], %p11: Tensor[(1, 64, 54, 54), float32], Primitive=1) -> Tensor[(1, 64, 54, 54), float32] {
    add(%p02, %p11) /* ty=Tensor[(1, 64, 54, 54), float32] */
  };
  %3 = %2(%1, meta[relay.Constant][0] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %4 = fn (%p01: Tensor[(1, 64, 54, 54), float32], %p1: Tensor[(1, 64, 54, 54), float32], Primitive=1) -> Tensor[(1, 64, 54, 54), float32] {
    add(%p01, %p1) /* ty=Tensor[(1, 64, 54, 54), float32] */
  };
  %5 = %4(%3, meta[relay.Const

In [11]:
relay.fromtext()

AttributeError: module 'tvm.relay' has no attribute 'fromtext'

In [ ]:
mod.import_from_std(mod1.astext())

In [1]:
import tvm
from tvm import relay
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *

In [3]:
pattern = is_op("nn.relu")(is_op("nn.conv2d")(wildcard(), wildcard()))

# A graph.
x = relay.var('input')
w = relay.var('weight')
conv2d = relay.op.nn.conv2d(x, w)
relu = relay.op.nn.relu(conv2d)
print(relu)
# free_var %x: Tensor[(1, 3, 224, 224), float32]
# free_var %w: Tensor[(3, 3, 3, 3), float32]
# %0 = nn.conv2d(%x, %w, padding=[0, 0, 0, 0]) /* ty=Tensor[(1, 3, 222, 222), float32] */;
# free_var %b: Tensor[(3), float32]
# nn.bias_add(%0, %b) /* ty=Tensor[(1, 3, 222, 222), float32] */




free_var %input;
free_var %weight;
%0 = nn.conv2d(%input, %weight, padding=[0, 0, 0, 0]);
nn.relu(%0)


In [4]:
# After partition.
print(pattern.partition(relu))
# free_var %x: Tensor[(1, 3, 224, 224), float32]
# free_var %w: Tensor[(3, 3, 3, 3), float32]
# free_var %b: Tensor[(3), float32]
# %1 = fn (%FunctionVar_0_0, %FunctionVar_0_1,
#          %FunctionVar_0_2, PartitionedFromPattern="nn.conv2d_nn.bias_add_") {
#   %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[0, 0, 0, 0]);
#   nn.bias_add(%0, %FunctionVar_0_2)
# };
# %1(%x, %w, %b)

free_var %input;
free_var %weight;
%1 = fn (%FunctionVar_0_0, %FunctionVar_0_1, PartitionedFromPattern="nn.conv2d_nn.relu_") {
  %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[0, 0, 0, 0]);
  nn.relu(%0)
};
%1(%input, %weight)


In [5]:
print(pattern.partition(relu, {'Composite': 'one_layer'}))
# free_var %x: Tensor[(1, 3, 224, 224), float32]
# free_var %w: Tensor[(3, 3, 3, 3), float32]
# free_var %b: Tensor[(3), float32]
# %1 = fn (%FunctionVar_0_0, %FunctionVar_0_1,
#          %FunctionVar_0_2, Composite="one_layer",
#                            PartitionedFromPattern="nn.conv2d_nn.bias_add_") {
#   %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[0, 0, 0, 0]);
#   nn.bias_add(%0, %FunctionVar_0_2)
# };
# %1(%x, %w, %b)

free_var %input;
free_var %weight;
%1 = fn (%FunctionVar_0_0, %FunctionVar_0_1, PartitionedFromPattern="nn.conv2d_nn.relu_", Composite="one_layer") {
  %0 = nn.conv2d(%FunctionVar_0_0, %FunctionVar_0_1, padding=[0, 0, 0, 0]);
  nn.relu(%0)
};
%1(%input, %weight)


In [6]:
def check(pre):
    conv = pre.args[0]
    return (conv.attrs.data_layout == "NCHW") and bool(conv.checked_type.shape[0] == 1)

pattern.partition(relu, check=check)

ValueError: Traceback (most recent call last):
  7: TVMFuncCall
  6: tvm::runtime::TypedPackedFunc<tvm::RelayExpr (tvm::relay::DFPattern, tvm::RelayExpr, tvm::runtime::Map<tvm::runtime::String, tvm::runtime::ObjectRef, void, void>, tvm::runtime::PackedFunc)>::AssignTypedLambda<tvm::relay::{lambda(tvm::relay::DFPattern, tvm::RelayExpr, tvm::runtime::Map<tvm::runtime::String, tvm::runtime::ObjectRef, void, void>, tvm::runtime::PackedFunc)#3}>(tvm::relay::{lambda(tvm::relay::DFPattern, tvm::RelayExpr, tvm::runtime::Map<tvm::runtime::String, tvm::runtime::ObjectRef, void, void>, tvm::runtime::PackedFunc)#3}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}::operator()(tvm::runtime::TVMArgs const, tvm::runtime::TVMRetValue) const
  5: tvm::relay::PartitionPattern(tvm::relay::DFPattern, tvm::RelayExpr, tvm::runtime::Map<tvm::runtime::String, tvm::runtime::ObjectRef, void, void>, tvm::runtime

In [71]:
class BatchnormCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        self.x = wildcard()
        self.var = wildcard()
        self.mean = wildcard()
        self.beta = wildcard()
        self.gamma = wildcard()
        self.eps = wildcard()

        self.pattern = self.gamma * (self.x - self.mean)/is_op("sqrt")(self.var + self.eps) + self.beta
        self.cnt = 0
        self.target = 2

    def callback(self, pre, post, node_map):
        if self.cnt != self.target:
            print(1)
            self.cnt += 1
        else :    
            # print(node_map)
            x = node_map[self.x][0]
            var = node_map[self.var][0]
            mean = node_map[self.mean][0]
            beta = node_map[self.beta][0]
            gamma = node_map[self.gamma][0]
            eps = node_map[self.eps][0]
            return relay.op.nn.batch_norm(x, gamma, beta, mean, var, epsilon = eps.data.numpy().item())[0]

# A graph of arithmetic operators that are functional equivalent to batch_norm.
x = relay.var('x')
var = relay.var('var')
mean = relay.var('mean')
beta = relay.var('beta')
gamma = relay.var('gamma')

JUNO = var + var + var + var + var + var + var 

BN = gamma * (x - mean)/relay.op.sqrt(var + var + relay.const(1e-5)) + beta
print(BN)
from tvm.relay.dataflow_pattern import rewrite
out = rewrite(BatchnormCallback(), BN)
# assert tvm.ir.structural_equal(out, relay.op.nn.batch_norm(x, gamma, beta, mean, var, epsilon = 1e-5)[0])

free_var %x;
free_var %mean;
free_var %gamma;
%0 = subtract(%x, %mean);
free_var %var;
%1 = add(%var, %var);
%2 = add(%1, 1e-05f);
%3 = multiply(%gamma, %0);
%4 = sqrt(%2);
%5 = divide(%3, %4);
free_var %beta;
add(%5, %beta)
1


TVMError: Traceback (most recent call last):
  10: TVMFuncCall
  9: tvm::runtime::TypedPackedFunc<tvm::RelayExpr (tvm::runtime::Array<tvm::relay::DFPatternCallback, void>, tvm::RelayExpr, tvm::IRModule)>::AssignTypedLambda<tvm::RelayExpr (*)(tvm::runtime::Array<tvm::relay::DFPatternCallback, void>, tvm::RelayExpr, tvm::IRModule)>(tvm::RelayExpr (*)(tvm::runtime::Array<tvm::relay::DFPatternCallback, void>, tvm::RelayExpr, tvm::IRModule), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}::operator()(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*) const
  8: tvm::relay::RewritePatterns(tvm::runtime::Array<tvm::relay::DFPatternCallback, void>, tvm::RelayExpr, tvm::IRModule)
  7: tvm::relay::PatternRewriter::Rewrite(tvm::runtime::Array<tvm::relay::DFPatternCallback, void> const&, tvm::RelayExpr const&)
  6: tvm::relay::PatternGrouper::GroupMatches(tvm::relay::DFPattern const&, tvm:

In [244]:
class JUNOCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)
        # self.x = wildcard()
        self.var1 = wildcard()
        self.var2 = wildcard()
        # self.var = wildcard()
        # self.meta = wildcard()
        self.var = relay.var("var")
        self.meta = relay.var("meta")
        self.padding = [1, 1, 1, 1]
        self.channels = 32 
        self.kernel_size = [3, 3]
        self.out_dtype = "int32"
        # self.var = wildcard()
        
        # self.mean = wildcard()
        # self.beta = wildcard()
        # self.gamma = wildcard()
        # self.eps = wildcard()

        # self.pattern = self.gamma * (self.x - self.mean)/is_op("sqrt")(self.var + self.eps) + self.beta
        # self.pattern = is_op(relay.op.nn.max_pool2d)
        # self.pattern = relay.op.nn.nn.max_pool2d(self.var1)
        # self.pattern = relay.op.nn.conv2d(
        #     self.var, 
        #     self.meta, 
        #     padding=self.padding, 
        #     channels=self.channels,
        #     kernel_size=self.kernel_size,
        #     out_dtype=self.out_dtype
        # )
        # self.pattern = is_op('nn.conv2d')(wildcard(), wildcard(), is_constant(), is_constant(), is_constant())
        self.pattern = is_op('nn.bias_add')(self.var1, self.var2)
        # self.pattern = is_op('max_pool2d')(self.var1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0] )
        self.cnt = 0
        self.target = 0

    def callback(self, pre, post, node_map):
        var1 = node_map[self.var1][0]
        var2 = node_map[self.var2][0]
        original = relay.nn.bias_add(var1, var2)

        if self.cnt != self.target:
            print(self.cnt, self.target)
            print("fuck")
            self.cnt += 1
            return original
        else:
            self.cnt += 1
            # return var1 * var2
            original = relay.nn.bias_add(var1, var2)
            cast_to_int8 = relay.cast(
                relay.clip(
                    relay.round(
                        relay.multiply(original, relay.const(16.0))
                    ), 
                    a_min=-127.0, a_max=127.0
                ),
                dtype="int8"
            )
            print(cast_to_int8)
            cast_to_float32 = relay.cast(
                relay.clip(
                    relay.right_shift(
                        relay.add(relay.cast(cast_to_int8, dtype='int32'), relay.const(512)),
                        relay.const(10)),
                    a_min=-127.0, a_max=127.0), 
                dtype="float32"
            )
            return cast_to_float32
            # return relay.cliprelay.right_shift(relay.add(relay.cast(
            #     relay.clip(
            #         relay.round(
            #             var1 * relay.const(16.0)
            #         ), 
            #         a_min=relay.const(-127.0), 
            #         a_max=relay.const(127.0)
            #     ), dtype="int8"), relay.const(512)), 10)

x1 = relay.var('x1')
x2 = relay.var('x2')
x3 = relay.var('x3')
x4 = relay.var('x4')
x5 = relay.var('x5')

JUNO = x1 + x2 + x3 + x4 + x5

print("===target relay===")
print(JUNO)
print("==================")
bc = JUNOCallback()
# print("target pattern")
# print(bc.pattern)
out = rewrite(JUNOCallback(), JUNO)
print("======OUT=========")
print(out)
print("==================")


===target relay===
free_var %x1;
free_var %x2;
%0 = add(%x1, %x2);
free_var %x3;
%1 = add(%0, %x3);
free_var %x4;
%2 = add(%1, %x4);
free_var %x5;
add(%2, %x5)
======OUT=========
free_var %x1;
free_var %x2;
%0 = add(%x1, %x2);
free_var %x3;
%1 = add(%0, %x3);
free_var %x4;
%2 = add(%1, %x4);
free_var %x5;
add(%2, %x5)


In [231]:
out = rewrite(JUNOCallback(), mod['main'])
out_body = rewrite(JUNOCallback(), mod['main'].body)


1 0
fuck
2 0
fuck
1 0
fuck
2 0
fuck


In [232]:
out

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = multiply(%1, 16f);
  %3 = round(%2);
  %4 = clip(%3, a_min=-127f, a_max=127f);
  %5 = cast(%4, dtype="int8");
  %6 = cast(%5, dtype="int32");
  %7 = add(%6, 512);
  %8 = right_shift(%7, 10);
  %9 = clip(%8, a_min=-127f, a_max=127f);
  %10 = cast(%9, dtype="float32");
  %11 = nn.max_pool2d(%10, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %12 = nn.conv2d(%11, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %13 = nn.bias_add(%12, %v_param_4);
  %14 = nn.max_pool2d(%13, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %15 =

In [233]:
mod['main'] = out
# mod['main'].body = out_body

In [230]:
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

print(out)

free_var %x1;
free_var %x2;
%0 = add(%x1, %x2);
free_var %x3;
%1 = add(%0, %x3);
free_var %x4;
%2 = add(%1, %x4);
free_var %x5;
add(%2, %x5)


In [213]:
mod['main'] = out

TVMError: Traceback (most recent call last):
  4: TVMFuncCall
  3: _ZNSt17_Function_handlerIFvN3
  2: tvm::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const [clone .isra.0]
  1: tvm::IRModuleNode::Add(tvm::GlobalVar const&, tvm::BaseFunc const&, bool)
  0: tvm::WarnIfMalformed(tvm::IRModule const&, tvm::relay::Function)
  File "/home/j/tvm-slicer/include/tvm/src/ir/module.cc", line 190
TVMError: 
---------------------------------------------------------------
An error occurred during the execution of TVM.
For more information, please see: https://tvm.apache.org/docs/errors.html
---------------------------------------------------------------

  Check failed: fv.size() == 0 (7 vs. 0) : Function:
fn () {
  free_var %input_1: Tensor[(1, 3, 256, 256), float32];
  free_var %v_param_1: Tensor[(16, 3, 3, 3), float32];
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  

In [242]:
# out_fun = tvm.IRModule.from_expr(out)
# mod['main'].body = out

target = 'cuda'
dev = tvm.cuda()

with tvm.transform.PassContext(opt_level=2):
    lib = relay.build(mod, target, params=params)


In [222]:
mod['main']

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(32, 16, 3, 3), float32], %v_param_4: Tensor[(32), float32], %v_param_5: Tensor[(64, 32, 3, 3), float32], %v_param_6: Tensor[(64), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %3 = nn.conv2d(%2, %v_param_3, padding=[1, 1, 1, 1], channels=32, kernel_size=[3, 3]);
  %4 = nn.bias_add(%3, %v_param_4);
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0]);
  %6 = nn.conv2d(%5, %v_param_5, padding=[1, 1, 1, 1], channels=64, kernel_size=[3, 3]);
  %7 = nn.bias_add(%6, %v_param_6);
  nn.max_pool2d(%7, pool_size=[2, 2], strides=[2, 2], padding=[0, 0, 0, 0])
}

In [208]:
import json
import pygraphviz as pgv

def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
            #A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(shape_size(json_data['attrs']['shape'][1][src[0]])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(shape_size(json_data['attrs']['shape'][1][node_idx])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')


In [243]:
show_graph(lib['get_graph_json'](), "quanadd_2")


In [202]:
import tensorflow as tf
model_keras = tf.keras.models.load_model('./very_simple_model.h5')

input_data = np.random.normal(0,1,(1,256,256,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


In [13]:
import numpy as np

In [15]:
x = relay.var('x')
rmultiply = relay.op.multiply(x, 1.0)
# r
# QN = 

TVMError: Traceback (most recent call last):
  2: TVMFuncCall
  1: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::relay::Call (tvm::RelayExpr, tvm::RelayExpr)>::AssignTypedLambda<tvm::relay::{lambda(tvm::RelayExpr, tvm::RelayExpr)#17}>(tvm::relay::{lambda(tvm::RelayExpr, tvm::RelayExpr)#17}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}>::_M_invoke(std::_Any_data const&, tvm::runtime::TVMArgs&&, tvm::runtime::TVMRetValue*&&)
  0: tvm::runtime::TVMMovableArgValueWithContext_::operator tvm::RelayExpr<tvm::RelayExpr>() const
  3: TVMFuncCall
  2: std::_Function_handler<void (tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*), tvm::runtime::TypedPackedFunc<tvm::relay::Call (tvm::RelayExpr, tvm::RelayExpr)>::AssignTypedLambda<tvm::relay::{lambda(tvm::RelayExpr, tvm::RelayExpr)#17}>(tvm::relay::{lambda(tvm::RelayExpr, tv

In [10]:
print(BN)

free_var %x;
free_var %mean;
free_var %gamma;
%0 = subtract(%x, %mean);
free_var %var;
%1 = add(%var, 1e-05f);
%2 = multiply(%gamma, %0);
%3 = sqrt(%1);
%4 = divide(%2, %3);
free_var %beta;
add(%4, %beta)


In [9]:
print(out)

free_var %x;
free_var %gamma;
free_var %beta;
free_var %mean;
free_var %var;
%0 = nn.batch_norm(%x, %gamma, %beta, %mean, %var);
%0.0
